In [1]:
!git clone https://github.com/GravityGravity/ASL-GestureNet.git

Cloning into 'ASL-GestureNet'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 79 (delta 35), reused 59 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 147.79 KiB | 5.68 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [6]:
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder


In [18]:
CSV_PATH = "/content/NEW_asl_train_dataset.csv"
df = pd.read_csv(CSV_PATH)
keypoint_columns = df.columns[4:]

In [19]:
def normalize_keypoints(keypoints):
    """
    keypoints: np.array shape (21, 2)
    returns: normalized keypoints as 1D vector (42,)
    """
    keypoints = np.array(keypoints, dtype=np.float32)
    wrist = keypoints[0]
    keypoints -= wrist

    max_val = np.abs(keypoints).max()
    if max_val > 0:
        keypoints /= max_val
    return keypoints.flatten()

def parse_keypoint(kp_str):
    return ast.literal_eval(kp_str)

features = []
for idx, row in df.iterrows():
    kp_list = [parse_keypoint(row[col]) for col in keypoint_columns]
    norm_kp = normalize_keypoints(kp_list)
    features.append(norm_kp)

features = np.array(features, dtype=np.float32)
print("Features shape after normalization:", features.shape)
print("First sample features:", features[0])


le = LabelEncoder()
labels = le.fit_transform(df['label'].values)

print("First 10 encoded labels:", labels[:10])
print("Classes:", le.classes_)

Features shape after normalization: (1819, 42)
First sample features: [ 0.          0.          0.13830592 -0.23021571  0.28054878 -0.48810416
  0.4690425  -0.6432188   0.666013   -0.7389616   0.02712212 -0.9185938
  0.5105348  -0.92765665  0.7792712  -0.79570127  0.9460401  -0.6878389
  0.         -0.95543987  0.5646302  -0.97636425  0.82932293 -0.8304167
  0.9951982  -0.688486    0.03251593 -0.90062493  0.59169567 -0.92159086
  0.84894794 -0.7735213   1.         -0.6315396   0.10904135 -0.7791894
  0.5780015  -0.7583296   0.7970938  -0.65965086  0.92574894 -0.56676745]
First 10 encoded labels: [0 0 0 0 0 0 0 0 0 0]
Classes: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h'
 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HandKeypointCNN(nn.Module):
    def __init__(self, num_classes=36):
        super().__init__()

        # Input: (B, 21, 2)
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm1d(128)

        self.dropout = nn.Dropout(0.3)

        # After convs: (B, 128, 21)
        self.fc1 = nn.Linear(128 * 21, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        # x: (B, 21, 2) → transpose to (B, 2, 21)
        x = x.transpose(1, 2)

        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))

        x = self.dropout(x)

        # Flatten
        x = x.flatten(start_dim=1)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [30]:
class KeypointDataset(Dataset):
    def __init__(self, features, labels):
        # features shape: (N, 42)
        self.X = torch.tensor(features, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]          # shape: (42,)
        x = x.view(21, 2)        # reshape → (21, 2)
        return x, self.y[idx]


dataset = KeypointDataset(features, labels)

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Train samples:", len(train_dataset), "Test samples:", len(test_dataset))

Train samples: 1637 Test samples: 182


In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = HandKeypointCNN().to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    print(f"\n=== Epoch {epoch+1}/{num_epochs} ===")

    for i, (X_batch, y_batch) in enumerate(tqdm(train_loader, desc="Training batches")):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        if i % 10 == 0:
            print(f"Batch {i} | Loss: {loss.item():.4f}")

    train_loss = running_loss / total
    train_acc = correct / total * 100
    print(f"Epoch {epoch+1} summary | Loss: {train_loss:.4f} | Accuracy: {train_acc:.2f}%")



=== Epoch 1/100 ===


Training batches:  15%|█▌        | 2/13 [00:00<00:00, 14.55it/s]

Batch 0 | Loss: 3.6140


Training batches: 100%|██████████| 13/13 [00:00<00:00, 17.45it/s]


Batch 10 | Loss: 2.4770
Epoch 1 summary | Loss: 2.9223 | Accuracy: 37.26%

=== Epoch 2/100 ===


...

Batch 10 | Loss: 0.7061
Epoch 99 summary | Loss: 0.7013 | Accuracy: 99.76%

=== Epoch 100/100 ===


Training batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batch 0 | Loss: 0.7040


Training batches: 100%|██████████| 13/13 [00:00<00:00, 18.08it/s]

Batch 10 | Loss: 0.7163
Epoch 100 summary | Loss: 0.7028 | Accuracy: 99.33%


In [32]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

test_acc = correct / total * 100
print(f"\nTest Accuracy: {test_acc:.2f}%")



Test Accuracy: 95.60%


In [33]:
MODEL_PATH = "asl_cnn_model.pth"
torch.save(model.state_dict(), MODEL_PATH)
print("Model saved to", MODEL_PATH)

Model saved to asl_cnn_model.pth
